## <strong> Import Header <strong>

In [11]:
import os
import json

import numpy as np
import matplotlib.pyplot as plt
from pycocotools.coco import COCO
from PIL import Image

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
DATASET_PATH = '/content/drive/MyDrive/dataset/'
TRAIN_LABEL_PATH = os.path.join(DATASET_PATH, 'label(polygon)_train.json')
TRAIN_IMAGES_PATH = os.path.join(DATASET_PATH, 'train')

## <strong> Draw Mask </strong>

In [9]:
with open(TRAIN_LABEL_PATH, "r") as file:
    json_file = json.load(file)

coco = COCO(os.path.join(TRAIN_LABEL_PATH))

loading annotations into memory...
Done (t=0.96s)
creating index...
index created!


In [ ]:
index = 100

img_file = os.path.join(TRAIN_IMAGES_PATH, json_file['images'][index]['file_name'])
img = Image.open(img_file)
img = np.array(img)

image_id = coco.getImgIds(imgIds=index)
image_infos = coco.loadImgs(image_id)[0]


ann_ids = coco.getAnnIds(imgIds=image_infos['id'])
anns = coco.loadAnns(ann_ids)
anns = sorted(anns, key=lambda idx : idx['area'], reverse=True) # 큰 사물부터 칠해야 하기 때문에 영역 크기의 내림차순으로 정렬한다.

cat_ids = coco.getCatIds()
cats = coco.loadCats(cat_ids)
categories_name = ['Normal']
categories = {}
for name in categories_name:
    categories[name]=0

mask = np.zeros((image_infos["height"], image_infos["width"]))
for i in range(len(anns)):
    class_num = anns[i]['category_id']  
    categories[categories_name[class_num-1]]+=1
    mask[coco.annToMask(anns[i]) == 1] = class_num              # coco.annToMask(anns[i])함수는 해당 annotation의 mask 라벨의 모든 픽셀과 해당 픽셀 내부를 1로 나타내준다.

print(f'Image Size: (height)-{image_infos["height"]}, (width)-{image_infos["width"]}')
print(categories)
fig, axs = plt.subplots(ncols=2, nrows=1, figsize=(16, 15))
axs[0].imshow(img)
axs[0].set_title('File Name: {}'.format(image_infos['file_name']))
axs[0].axis('off')
axs[1].imshow(mask)
axs[1].set_title('Mask Name: {}'.format(image_infos['file_name']))
axs[1].axis('off')
plt.show()